In [1]:
import os
import pandas as pd
import OAI

%load_ext autoreload 

In [2]:
STEP1 = "Acting as a Futurist, process the following text as a signal in a TABLE, first column a short 5–7 word summary of the signal, second column what kind of change is this (from what to what) 5–10 word summary, third column what might be different in 10 years time 5–10 word summary, fourth column What’s one driving force, or motivation, behind this change? 5–10 word summary:\n\n"

STEP2 = "Process the following text, and give back only the top 10 most relevant keywords for the text, as a Python list (looking like KEYWORDS = ['keyword1', 'keyword2']). Then, on a new line, provide a list of the top three themes or categories the text belongs to (looking like THEMES = ['theme1', 'theme2']). On another new line, add a paragraph starting with 'Summary:'  which  summarizes the text in 4 to 5 sentences. \n\n"

STEP3 = "Process the following text, and give this text a title, which should not be longer than 6 words, in the form 'TITLE: title of the article'.\n\n"


In [3]:
df = pd.read_parquet('articles.parquet.gzip')
df = df[(df.LEN > 1500) & (df.LEN < 30000)]
df

,file_name,content,LEN
0,58612e45effab1e47df9a86b14dfae85,Discover more from The Generalist\nLife in a K...,22376
1,115b59fc3f0d7b148482545adb1a8038,Google DeepMind researchers have discovered 2....,3950
2,fab122d29aed97045e0cc1ea77bdef44,The Responsible AI Institute\nNavigating Organ...,13194
3,8b7b3c515859db4c0f6537824019cb7b,"PANAMA CITY, Oct 31 (Reuters) – The Panama Can...",2049
4,338c551fe29e9fb73aa8d412dc77788e,"By Rishad Tobaccowala\nDecades ago, the Book o...",5082
...,...,...,...
1591,edbb7336dfb26098bcc966ca17c074b9,It was time to consider calling it a night. Sh...,15656
1592,3fbb11d6e949d1e662aa6a146bb6cda0,China’s growing number of insomniacs are turni...,2859
1593,e8b7faf708568f9f39abe04b778c4631,I would say my efficiency is up ~20% since sta...,14337
1594,b79a2baa87b68283198416791b93bce4,The U.S. government has restricted sales of Nv...,2125


In [4]:
h = OAI.Helper("local") 

In [5]:
for ix, row in df.sample(len(df)).iterrows():
    ID, txt = row.file_name, row.content
    CACHE1 = ".cache/"+str(ID)
    CACHE2 = ".cache_keywords/"+str(ID)
    CACHE3 = ".cache_title/"+str(ID)
    print(row.file_name)
    if ix < 100000:
        if not os.path.isfile(CACHE1):
            table = h.ask(STEP1,txt,src="localSeedsStep1")
            with open(CACHE1, 'w', encoding='utf-8') as f:
                f.write(table)
        if not os.path.isfile(CACHE2):
            table = h.ask(STEP2,txt,src="localSeedsStep2")
            with open(CACHE2, 'w', encoding='utf-8') as f:
                f.write(table)
        if not os.path.isfile(CACHE3):
            table = h.ask(STEP3,txt,src="localSeedsStep3")
            with open(CACHE3, 'w', encoding='utf-8') as f:
                f.write(table)       

3fbb11d6e949d1e662aa6a146bb6cda0
cefe931c301344e0c80882be9c497dc6
bf6fa37b7bc836eca6faa20578b2718e
38548b322684d1a81aa9ef61c7312cee
40a7d90f20bae6180001e65049ae853b
3d4aee592248f940acdac6ffab5ec4aa
06aa7cdbd43c9b2c38bfa2f44b46d215
d5c399872ea4d28def48f650503511a0
d7e3c94e1140ebf6647a4d77db5c4c5e
633655a8e8a99ad217f019928a2b8dc1
46c1f73cbd3e48207a7b612fe8678cd3
ea69e4f0817d3b814cde232ed2e163fe
9e74c32425eab87ba36167bfe05c8f2c
213ff4d78c8040377180b7f7b5b13d4d
1bc6c11e394022139f8af61a7554a7ec
758ced101c96751a5fadc7dc7dbc238d
fc725e773fd7ad77d91e2c903607ef36
b396d58ff64cbfdfe136c278b263ee5a
d28d19739c279a21eb19cac312e67bea
a38960ed1dcfd200d74ebd16e92e4151
a0e48b4839649f3f833b1a9b21d0dfbc
01dd20372573227317c7126faacfec9b
3955987c235652b24c7507a6999cc80e
f87bcfb74a4a1db0ac38bd573144fd59
d64c4666be4d84f147f57055b74daa32
add3440e907f0e4e9cbb7fbbdd544a90
d943c22736f43f56cebc890da1718595
2ef1681fbde001909e631d18aad43f79
2288acc14dd8aab5ef245655d253626f
5faa4668aa08755679e2956ba96ecb18
44ddf53ef2

618c3b31c0c4ef4e6b764a04f0f09601
587c21780f97f331b4d2c859240e09e9
49e0af19d90eda0f809a16b24ee8cbee
e69dcd964ee3d865e155042c8fb38cc5
b1b8a02e5d2badf1881b81741a66ba86
7b316ebe449187b79e519a8c6d12a2cd
06ca8cfe66a96c99e2830d74b7493668
e623b9d82816485d864a5faeb2f0d15b
8bb1226617cfc9b177ec5976f3716680
2feb921909e9c0903f0cd9af69456416
447aaddba1b3b4ac319e14628e6ed8f2
8fb8dc2edfbbb62dc95ff6e7ee1cae5c
be72c306267d04fbf21e2cd6051c66bd
074f7e40545f4b8edc9bdefa00b02f7b
ccbfba09df6fed1b88325852e7c3e80d
42029469086ed8b0defa5120ec35459e
9ea825ae2ff757c9a593882acf36334d
e1fbb09ec5e66a8a6d4eff2126eefb40
697e711fcf671aa443cffb8f3d8a7b0e
8bff520dc945204951b32a6c4b10571e
07f58082de6736a23fae6143ab961f07
4905ff1ba8fbf6dc6f9916d05ddcad46
4b66dbaa2bbc249bfcccbb7e27adbde5
997c962feb825d066ca4dc6e4742e8a9
f6a4f1f9e66f99bd101dee7d6a800855
27a3a94222f061917b211c94d1d3ee92
a0aa4aeba716d3559002da147889760a
6d08036a53cd3136ac5a71006b4ca40e
ed6be3a2a502c8a0bedd63d8ab1c8d9e
60d708d49e171255bc45464e0b5e6a6a
60c07fb2fe

c3876ee8e1b4e0d9e38edf15caaa5521
bccb58e39d04eb0ca494d80ca20e6a67
364fecbe1c46f164e48d52c279422351
763a73de80e3ade3d3a068da6ce48182
2e15bb8c640aadb6d9022cf48f446954
6fd7033b884ae982dac41bad4396abb5
4140e375969a4396e244eb285ef19caa
52ec2cf0aebdc7af56249f1702652ebe
293548cd25437541d9a043d6f9e07d47
3c8e50874cd8552a38f5adab70ccd2b7
40c2db3bc4702bff2612a3faba89097a
6335d1cfa75abf9650361efd7b529149
a764338d3c1cf114377e3f619e246289
c2544a0a5298530b102bb76ca874b3d5
d3ca03ec9e48516fa3d82a78cf2d61c0
5554792b473e148108ccb7b38a1d82c0
73f1352a504ec78d7c92fa6f9b7c6922
1739f639d5bfca8e60d7750e29cc6ab3
a875869ad6d7b974664d20f35dc8d83c
ba66ec5438b7d5899fbfa973e82d66ca
bf4505c73a7c8cd0d84f15d3384b0433
fec8d34946df9092e1ade8823b1d5f6d
faad51d1f885af1db9dee5e5ce52b46e
589c2c554219a6d4ae1ad0c9d1d75a21
ccc68b31f2bbee79789078f87c6b0c9d
8c3af57e1a9f1b3f778f7b3cefcd6318
dd9fc6f91eeb99cb289ba2e9f8af02f3
4ef8cee75515da28d5c6ad986ee83030
ce76b9ab29be2d9ff9fe34c52b5e396b
d44d0c046d2ee3330a2d40cfccdfc0d3
cdc3436dbc

In [6]:
import io
def txtRead(filename):
    with io.open(filename,'r',encoding='utf8') as f:
        txt= f.read()
    return txt

In [7]:
import glob

In [8]:
SEEDS = list(glob.glob(".cache_title/*"))

allSeeds = []
for seed in SEEDS: 
    try:
        ID = seed.split("/")[-1]
        txt = txtRead(seed) 
        txt = txt.replace("\n\n","\n")
        SUMMARY = txt.replace("TITLE:","").split("\n")[0].strip()
        allSeeds.append([ID,SUMMARY])
    except:
        print("Error with:",seed)

In [9]:
DDTitle = pd.DataFrame(allSeeds)
DDTitle.columns = ["src","title"]
DDTitle.to_parquet("titles.parquet.gzip",compression='gzip')
DDTitle

,src,title
0,ea74b217d36ec570bb432fb5aa679090,The Impact of AI on Education
1,841546c0efc4c82f0aabc545a47a09e1,Researchers Uncover Backdoor in Encrypted Radi...
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,Hack-a-Sat: Researchers Launch Sandbox Satelli...
3,58612e45effab1e47df9a86b14dfae85,The Challenges of Regulating Artificial Intell...
4,7ff6239e13a43e856fa36887902c5ede,Building a Knowledge Graph with Chat-GPT and P...
...,...,...
1191,edbb7336dfb26098bcc966ca17c074b9,The Radical Power of Hanging Out
1192,3fbb11d6e949d1e662aa6a146bb6cda0,China's Insomniacs Turn to Sleeping Assistants
1193,e8b7faf708568f9f39abe04b778c4631,The Impact of ChatGPT on Efficiency and Inform...
1194,b79a2baa87b68283198416791b93bce4,US Restricts Nvidia GPU Sales to Thwart China'...
